# libraries

In [15]:
import pandas as pd
import json
import numpy as np
import math


In [21]:
def compose_coordinate(latitude, longitude):
    composed_coordinate = '['+ str(longitude)+ ','+ str(latitude)+ ']'
    return composed_coordinate

# def haversine_distance(coord1, coord2):
#     R = 6371.0  # Earth's radius in km
#     lat1 = math.radians(coord1[0])
#     lon1 = math.radians(coord1[1])
#     lat2 = math.radians(coord2[0])
#     lon2 = math.radians(coord2[1])
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

#     distance = R * c
#     return distance

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance


# import data

In [3]:
total_node = pd.read_csv("./total_node.csv")
total_link = pd.read_csv("./link_b_h.csv")
print("노드의 크기:",total_node.shape)
print("링크의 크기:",total_link.shape)

노드의 크기: (461110, 2)
링크의 크기: (150914, 17)


### 노드 반올림

In [4]:
total_node['경도반올림'] = total_node['경도'].round(9)
total_node['위도반올림'] = total_node['위도'].round(9)

### 중복 노드 제거 및 링크 넘버링

In [5]:
total_node.drop_duplicates(keep='first')

,경도,위도,경도반올림,위도반올림
0,126.896267,37.258775,126.896267,37.258775
1,126.896269,37.258798,126.896269,37.258798
2,126.994921,37.253077,126.994921,37.253077
3,126.994904,37.253112,126.994904,37.253112
4,127.002715,37.254127,127.002715,37.254127
...,...,...,...,...
461102,127.001000,37.230355,127.001000,37.230355
461103,127.001046,37.230343,127.001046,37.230343
461107,127.025045,37.235468,127.025045,37.235468
461108,127.025000,37.235515,127.025000,37.235515


In [6]:
total_node.duplicated(['경도반올림','위도반올림']).sum()

108609

In [7]:
total_node.duplicated(['경도','위도']).sum()

108609

In [8]:
total_node.head()

,경도,위도,경도반올림,위도반올림
0,126.896267,37.258775,126.896267,37.258775
1,126.896269,37.258798,126.896269,37.258798
2,126.994921,37.253077,126.994921,37.253077
3,126.994904,37.253112,126.994904,37.253112
4,127.002715,37.254127,127.002715,37.254127


In [9]:
total_link.head()

,노드1,노드2,거리,경찰서,공원,cctv,가로등,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설
0,"[126.93270132291545, 37.295702765114804]","[126.93264903971692, 37.29569846547432]",0.004649,5.159311,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[126.93264903971692, 37.29569846547432]","[126.93259675660585, 37.29569415499454]",0.004650,5.163848,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[126.93259675660585, 37.29569415499454]","[126.93258655212853, 37.29569331105071]",0.000908,5.166560,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"[126.93258655212853, 37.29569331105071]","[126.93254917670525, 37.295690667471995]",0.003319,5.168617,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"[126.93254917670525, 37.295690667471995]","[126.9325279527859, 37.29568872688521]",0.001890,5.171156,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
#link의 weight을 만들었다. 나중에 거리와 위험도를 종합적으로 따져 weight값을 정할 것이고 현재는 그냥 거리값만 적용시켰다.
total_link.insert(2, 'weight', total_link['거리'].copy())



### 합쳐진 경도 위도 나누기

In [11]:
templink = pd.DataFrame( columns=['노드1경도', '노드1위도','노드2경도','노드2위도'])
for i in range(total_link.shape[0]):
    nodedata = json.loads(total_link['노드1'][i])
    nodedata.extend(json.loads(total_link['노드2'][i]))
    templink.loc[i] = nodedata

templink.head()


,노드1경도,노드1위도,노드2경도,노드2위도
0,126.932701,37.295703,126.932649,37.295698
1,126.932649,37.295698,126.932597,37.295694
2,126.932597,37.295694,126.932587,37.295693
3,126.932587,37.295693,126.932549,37.295691
4,126.932549,37.295691,126.932528,37.295689


### 반올림

In [12]:
#optional

templink['노드1경도'] = templink['노드1경도'].round(9)
templink['노드1위도'] = templink['노드1위도'].round(9)
templink['노드2경도'] = templink['노드2경도'].round(9)
templink['노드2위도'] = templink['노드2위도'].round(9)

### 노드 인덱스 연결

In [13]:
#optional

node1connect = []
node2connect = []

for i in range(templink.shape[0]):
    
    contemp = total_node.loc[(total_node['위도반올림'] == templink['노드1위도'][i]) & (total_node['경도반올림'] == templink['노드1경도'][i])]
    try:
        node1connect.append(contemp.index[0])
    except:
        print("{}번째 데이터 [{},{}] not in node".format(i, templink['노드1위도'][i], templink['노드1경도'][i]))
        node1connect.append(None)
    # if len(contemp.index) > 1:
    #     print("{}번째 데이터 [{},{}] duplicate node".format(i, templink['노드1위도'][i], templink['노드1경도'][i]))

    contemp = total_node.loc[(total_node['위도반올림'] == templink['노드2위도'][i]) & (total_node['경도반올림'] == templink['노드2경도'][i])]
    try:
        node2connect.append(contemp.index[0])
    except:
        print("{}번째 데이터[{},{}] not in node".format(i, templink['노드2위도'][i], templink['노드2경도'][i]))
        node2connect.append(None)
    # if len(contemp.index) > 1:
    #     print("{}번째 데이터 [{},{}] duplicate node".format(i, templink['노드2위도'][i], templink['노드2경도'][i]))

templink.insert(0, 'idx노드1', node1connect)
templink.insert(1, 'idx노드2', node2connect)


21663번째 데이터[37.326324665,126.957343228] not in node
21664번째 데이터 [37.326324665,126.957343228] not in node


## 임시로 위 2개 데이터 삭제할것

In [14]:
total_link = pd.concat([templink, total_link], axis=1)

total_link = total_link.drop([21663, 21664])


total_link.head()


,idx노드1,idx노드2,노드1경도,노드1위도,노드2경도,노드2위도,노드1,노드2,weight,거리,...,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설
0,71076.0,71077.0,126.932701,37.295703,126.932649,37.295698,"[126.93270132291545, 37.295702765114804]","[126.93264903971692, 37.29569846547432]",0.004649,0.004649,...,0,0,0,0,0,0,0,0,0,0
1,71077.0,71078.0,126.932649,37.295698,126.932597,37.295694,"[126.93264903971692, 37.29569846547432]","[126.93259675660585, 37.29569415499454]",0.004650,0.004650,...,0,0,0,0,0,0,0,0,0,0
2,71078.0,71079.0,126.932597,37.295694,126.932587,37.295693,"[126.93259675660585, 37.29569415499454]","[126.93258655212853, 37.29569331105071]",0.000908,0.000908,...,0,0,0,0,0,0,0,0,0,0
3,71079.0,71080.0,126.932587,37.295693,126.932549,37.295691,"[126.93258655212853, 37.29569331105071]","[126.93254917670525, 37.295690667471995]",0.003319,0.003319,...,0,0,0,0,0,0,0,0,0,0
4,71080.0,71081.0,126.932549,37.295691,126.932528,37.295689,"[126.93254917670525, 37.295690667471995]","[126.9325279527859, 37.29568872688521]",0.001890,0.001890,...,0,0,0,0,0,0,0,0,0,0


# 시작점 입력

In [162]:
dep_lat, dep_lon = 37.27711051820087, 127.04079216093226
des_lat, des_lon = 37.27559414340661, 127.04224810694416




# dep_lat = 37.28293695757347
# dep_lon = 127.0434230405271
# des_lat = 37.28312042096528
# des_lon = 127.04739397575078

# Get Nearest Node
출발점과 도착점의 가장 가까운 노드를 찾는 기능이다. 

In [163]:
def get_Nearest_Node(latitude, longitude):
    length_filter = 0.0003
    min_distance = 1e9
    nearest_lat = 0
    nearest_lon = 0
    nearest_idx = 0
    
    filtered_df = total_node.loc[(total_node['위도'] > latitude - length_filter) & (total_node['위도'] < latitude + length_filter) & (total_node['경도'] < longitude - length_filter) & (total_node['경도'] < longitude + length_filter)]
    for index, row in filtered_df.iterrows():

        cur_node_lat = row['위도']
        cur_node_lon = row['경도']
        cur_distance = haversine_distance(latitude, longitude, cur_node_lat, cur_node_lon)

        if cur_distance < min_distance:
            min_distance = cur_distance
            nearest_lat = cur_node_lat
            nearest_lon = cur_node_lon
            nearest_idx = index

    #return filtered_df
    return nearest_idx, nearest_lat, nearest_lon
    

start_idx, start_lat, start_lon = get_Nearest_Node(dep_lat, dep_lon)
end_idx, end_lat, end_lon = get_Nearest_Node(des_lat, des_lon)


In [164]:
print(start_idx, end_idx)

386223 84635


# 다익스트라 알고리즘

In [165]:
num_rows = total_node.shape[0]
dijkstra_df = {'col1': [np.inf] * num_rows, 'col2': [False] * num_rows, 'col3': [np.nan] * num_rows}
dijkstra_df = pd.DataFrame(data=dijkstra_df)
dijkstra_df.columns=['length', 'visited', 'route']

In [166]:
# start_idx = 33568
# #start_idx = 19265
# end_idx = 4

In [167]:
dijkstra_df.loc[start_idx, "length"] = 0
dijkstra_df.at[start_idx, "route"] = start_idx
now = start_idx

#num_visited = 0
while(True):

    unvisited_dijkstra_df = dijkstra_df.loc[(dijkstra_df['visited']  == False) & (dijkstra_df['length']  != np.inf)]
    if unvisited_dijkstra_df.empty:
        print('unvisited empty')
        break

    unvisited_dijkstra_df.sort_values(by=['length'])
    now = unvisited_dijkstra_df.index[0]
    #num_visited += 1
    #print("visited", now, "total visit:", num_visited)
    now_length = dijkstra_df.loc[start_idx, "length"]
    
    dijkstra_df.loc[now, "visited"] = True
    #dijkstra_df.loc[now, "route"] = [start_idx]

    now_connected_link = total_link.loc[(total_link['idx노드1']  == now)]

    for index, row in now_connected_link.iterrows():
        search_node_num = row['idx노드2']
        search_node_weight = row['weight']
        #print(now, ' is connected to ', search_node_num)

        if dijkstra_df.loc[search_node_num, "visited"] == True:
            continue
        
        if dijkstra_df.loc[search_node_num, "length"] > (now_length + search_node_weight):
            dijkstra_df.loc[search_node_num, "length"] = now_length + search_node_weight
            # templist = dijkstra_df.loc[now, "route"].copy()
            # templist.append(now)
            # print(templist)
            dijkstra_df.at[search_node_num, "route"] = now
            #print("modified length of ",search_node_num)


    now_connected_link = total_link.loc[(total_link['idx노드2']  == now)]
    for index, row in now_connected_link.iterrows():
        search_node_num = row['idx노드1']
        search_node_weight = row['weight']
        #print(now, ' is connected to ', search_node_num)

        if dijkstra_df.loc[search_node_num, "visited"] == True:
            continue
        
        if dijkstra_df.loc[search_node_num, "length"] > (now_length + search_node_weight):
            dijkstra_df.loc[search_node_num, "length"] = now_length + search_node_weight
            # templist = dijkstra_df.loc[now, "route"].copy()
            # templist.append(now)
            # print(templist)
            dijkstra_df.at[search_node_num, "route"] = now
            #print("modified length of ",search_node_num)


unvisited empty


# 최적화 방법

특정 사각형 내에서만 link를 제한하여 그 안에서만 경로를 돌린다

dictionary등으로 변경해본다

첫번째 경로 찾으면 끊는다? 이게 과연 최단일까?

# 결과 및 경로

In [168]:
route_list = [[des_lat,des_lon]]
cur_idx = end_idx


while(True):
    
    cur_lat = total_node.loc[cur_idx, "위도"]
    cur_lon = total_node.loc[cur_idx, "경도"]
    route_list.append([cur_lat, cur_lon])
    if cur_idx == start_idx:
        break
    
    cur_idx = dijkstra_df.loc[cur_idx, "route"]

route_list.append([dep_lat, dep_lon])


In [169]:
route_list

[[37.27559414340661, 127.04224810694416],
 [37.27537681670567, 127.04190727247848],
 [37.27532176874983, 127.04207085685998],
 [37.27530643387199, 127.04211643392372],
 [37.27546965314273, 127.0422055656462],
 [37.27591319348564, 127.04244778152082],
 [37.275927256040845, 127.04245545688902],
 [37.27598576032505, 127.04248071240649],
 [37.27600806717677, 127.04241333410408],
 [37.27611122200804, 127.04210173919992],
 [37.27619472621016, 127.04183127820328],
 [37.27619961463696, 127.0418175819563],
 [37.27621930375612, 127.04176238780047],
 [37.27625301631317, 127.04165074559926],
 [37.27625700222419, 127.0416386272092],
 [37.27641034209876, 127.04117287770228],
 [37.27651638352273, 127.04084222829093],
 [37.27654541948208, 127.0407516776704],
 [37.27657281604443, 127.04065818612892],
 [37.27658259466218, 127.04062481500436],
 [37.27686523076376, 127.03975708237924],
 [37.27711051820087, 127.04079216093226]]

In [170]:
now_connected_link = total_link.loc[(total_link['idx노드1']  == now)]
for i in range(now_connected_link.shape[0]):
    

SyntaxError: incomplete input (2211525418.py, line 3)

In [ ]:
#get_Nearest_Node 함수가 완성되어 가장 가까운 노드가 여기라고 가정하자

start_lat = 37.2793947321759
start_lon = 127.016818947031
end_lat = 37.2795395918437
end_lon = 127.016815561923



#이거말고 start idx end idx 해야 편할듯

In [ ]:
tempstart_df = total_link.loc[(total_link['노드1']  == start_coordinate)]# & (total_node['위도'] < latitude + length_filter) & (total_node['경도'] < longitude - length_filter) & (total_node['경도'] < longitude + length_filter)]


tempstart_df

,노드1,노드2,weight,거리,경찰서,공원,cctv,가로등,골목상권,공공체육시설,공동주택,노인복지시설,녹지,보안등,비상급수시설,지역아동센터,청소년공부방,어린이놀이시설


In [ ]:


filnode_lat = total_node['위도'][458109]
filnode_lon = total_node['경도'][458109]


filnode_df = total_node.loc[(total_node['위도'] == filnode_lat) & (total_node['경도'] == filnode_lon)]
filnode_df

fillink_df

,경도,위도
458108,127.039858,37.237389
458109,127.039858,37.237389


In [ ]:
#링크와 노드 데이터가 매칭되는지 확인했던 함수이며 안쓰일것


for i in range(461109):
    filnode_lat = total_node['위도반올림'][i]
    filnode_lon = total_node['경도반올림'][i]
    
    fillink_df = total_link.loc[(total_link['노드1위도']  == filnode_lat) & (total_link['노드1경도']  == filnode_lon)]
    if fillink_df.shape[0] > 1:
        print(fillink_df.shape, i)
    if i % 10000 == 0:
        print(i)

0
(2, 22) 799
(2, 22) 2157
(2, 22) 2500
(2, 22) 6696
(3, 22) 7321
(2, 22) 9026
(2, 22) 9304
10000
(2, 22) 19265
(2, 22) 19290
(2, 22) 19408
20000
(2, 22) 23065
(2, 22) 25223
(2, 22) 25795
(2, 22) 29056
(2, 22) 29963
30000
(2, 22) 31996
(2, 22) 33087
(2, 22) 33095
(2, 22) 33126
(2, 22) 33320
(2, 22) 33568
(2, 22) 33849
(2, 22) 33857
(2, 22) 33979
(2, 22) 34039
(2, 22) 34970
(2, 22) 35015
(2, 22) 35081
(2, 22) 35157
(4, 22) 35707
(3, 22) 35997
(2, 22) 36233
(3, 22) 36269
40000
(2, 22) 40006
(2, 22) 40955
(2, 22) 45520
(2, 22) 47462
(2, 22) 48495
50000
(2, 22) 50713
(2, 22) 50720
(2, 22) 50740
(2, 22) 50790


KeyboardInterrupt: 

In [ ]:
for i in range(461109):
    filnode_lat = total_node['위도'][i]
    filnode_lon = total_node['경도'][i]
    

127.016819
